In [1]:
import classiq
classiq.authenticate()

c:\Users\Aravind\college\quantum\venv\Lib\site-packages\classiq\_internals\authentication\token_manager.py:101: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [5]:
from classiq import *

@qfunc
def main(x: Output[QNum], y: Output[QNum]) -> None:
    """
    Quantum circuit simulating key distribution
    """
    # Allocate quantum numbers for qubits and bases
    allocate(3, x)  # Qubits
    allocate(3, y)  # Bases
    
    # Create superposition for bases
    hadamard_transform(x)
    hadamard_transform(y)

# Create the quantum model
qmod = create_model(main)

# Synthesize the circuit
qprog = synthesize(qmod)

# Visualize the circuit
show(qprog)

# Execute the quantum program
job = execute(qprog)

# Print results
print("Quantum Key Distribution Simulation:")
results = job.get_sample_result().parsed_counts
print(results)

Opening: https://platform.classiq.io/circuit/2unpI4a85jhPFFBN30yrIsOk3WI?login=True&version=0.70.0
Quantum Key Distribution Simulation:
[{'x': 3, 'y': 3}: 49, {'x': 2, 'y': 4}: 48, {'x': 4, 'y': 5}: 45, {'x': 6, 'y': 7}: 44, {'x': 4, 'y': 4}: 44, {'x': 1, 'y': 2}: 43, {'x': 0, 'y': 6}: 42, {'x': 0, 'y': 7}: 40, {'x': 2, 'y': 6}: 39, {'x': 3, 'y': 1}: 39, {'x': 3, 'y': 6}: 38, {'x': 2, 'y': 7}: 37, {'x': 1, 'y': 5}: 37, {'x': 5, 'y': 6}: 36, {'x': 4, 'y': 6}: 36, {'x': 7, 'y': 0}: 35, {'x': 4, 'y': 2}: 35, {'x': 6, 'y': 6}: 35, {'x': 1, 'y': 6}: 34, {'x': 2, 'y': 2}: 34, {'x': 1, 'y': 7}: 34, {'x': 7, 'y': 2}: 33, {'x': 6, 'y': 4}: 33, {'x': 3, 'y': 4}: 33, {'x': 6, 'y': 2}: 33, {'x': 4, 'y': 1}: 33, {'x': 7, 'y': 6}: 33, {'x': 2, 'y': 0}: 32, {'x': 2, 'y': 5}: 32, {'x': 5, 'y': 3}: 32, {'x': 2, 'y': 3}: 32, {'x': 0, 'y': 4}: 31, {'x': 7, 'y': 5}: 31, {'x': 1, 'y': 1}: 31, {'x': 7, 'y': 7}: 30, {'x': 5, 'y': 1}: 30, {'x': 1, 'y': 3}: 30, {'x': 0, 'y': 5}: 30, {'x': 7, 'y': 3}: 29, {'x':

In [ ]:
from classiq import *

@qfunc
def generate_alice_qubits(x: Output[QArray[8]], bases: Output[QArray[8]]) -> None:
    """
    Generate Alice's qubits with random states and bases.
    """
    allocate(x)      # Allocate 8 qubits for Alice's key
    allocate(bases)  # Allocate 8 qubits for Alice's bases
    
    # Randomize Alice's bases
    apply(H, bases)  
    
    # Prepare Alice's qubits based on random bases
    repeat(8, lambda i: control(bases[i], lambda: apply(H, x[i])))

@qfunc
def measure_bob_qubits(x: Input[QArray[8]], bases: Output[QArray[8]], results: Output[CArray[8]]) -> None:
    """
    Bob measures Alice's qubits using his randomly chosen bases.
    """
    allocate(bases)  # Allocate 8 qubits for Bob's measurement bases
    allocate(results)  # Allocate 8 classical bits for Bob's results

    # Bob generates random measurement bases
    apply(H, bases)

    # Apply Hadamard to qubits where Bob's basis is 1
    repeat(8, lambda i: control(bases[i], lambda: apply(H, x[i])))

    # Measure the entire QArray at once
    measure_qarray(x, results)

def bb84_key_distribution():
    """
    Simulate BB84 Quantum Key Distribution Protocol.
    """
    # Define the quantum function
    @qfunc
    def main(
        alice_states: Output[QArray[8]], 
        alice_bases: Output[QArray[8]], 
        bob_bases: Output[QArray[8]], 
        bob_results: Output[CArray[8]]
    ) -> None:
        # Alice generates qubits
        generate_alice_qubits(alice_states, alice_bases)
        
        # Bob measures the qubits
        measure_bob_qubits(alice_states, bob_bases, bob_results)
    
    # Create and synthesize the quantum model
    qmod = create_model(main)
    qprog = synthesize(qmod)
    
    # Execute the quantum program
    job = execute(qprog)
    result = job.get_sample_result()
    
    # Parse results
    alice_states = result.parsed_outputs['alice_states']
    alice_bases = result.parsed_outputs['alice_bases']
    bob_bases = result.parsed_outputs['bob_bases']
    bob_results = result.parsed_outputs['bob_results']
    
    # Perform key sifting (only matching bases contribute to the shared key)
    shared_key = [bob_results[i] for i in range(8) if alice_bases[i] == bob_bases[i]]
    
    print("Alice's Original States:", alice_states)
    print("Alice's Bases:", alice_bases)
    print("Bob's Bases:", bob_bases)
    print("Bob's Measurement Results:", bob_results)
    print("Shared Key after Sifting:", shared_key)
    print("Shared Key Length:", len(shared_key))

# Run the BB84 simulation
bb84_key_distribution()


AttributeError: 'QBit' object has no attribute 'h'